# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - C (hand-written)
    - C (hand-written with -ffast-math)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [94]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.8163986267625749 
 0.03247267322850034
 0.29173184282103537
 0.5481116964772641 
 0.48562095725891674
 0.6226185079152236 
 0.802436584117411  
 0.31003662772718377
 0.35479112128957624
 0.7642653236882644 
 0.44653367590415827
 0.9337431922966564 
 0.646275585292283  
 ⋮                  
 0.502978833771003  
 0.2578317944719539 
 0.6031130492923571 
 0.40820947328633594
 0.9475854333100218 
 0.3228053583247028 
 0.9495688212008535 
 0.48469555713081314
 0.1956668119662135 
 0.8489466973139901 
 0.3461309152789154 
 0.9174772429988314 

In [95]:
sum(a)

5.001145501676621e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [96]:
@time sum(a)

  0.007702 seconds (5 allocations: 176 bytes)


5.001145501676621e6

In [97]:
@time sum(a)

  0.007598 seconds (5 allocations: 176 bytes)


5.001145501676621e6

In [98]:
@time sum(a)

  0.007807 seconds (5 allocations: 176 bytes)


5.001145501676621e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [99]:
using Pkg
Pkg.add("BenchmarkTools")

 Resolving package versions...
  Updating `C:\Users\Guru\.juliapro\JuliaPro_v1.0.5-2\environments\v1.0\Project.toml`
 [no changes]
  Updating `C:\Users\Guru\.juliapro\JuliaPro_v1.0.5-2\environments\v1.0\Manifest.toml`
 [no changes]


In [100]:
using BenchmarkTools

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [ ]:
using Libdl

In [101]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

Base.IOError: IOError: could not spawn `gcc -fPIC -O3 -msse3 -xc -shared -o 'C:\Users\Guru\AppData\Local\Temp\jl_BDE1.tmp.dll' -`: no such file or directory (ENOENT)

In [102]:
c_sum(a)

UndefVarError: UndefVarError: c_sum not defined

In [103]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

UndefVarError: UndefVarError: c_sum not defined

In [104]:
c_sum(a) - sum(a)

UndefVarError: UndefVarError: c_sum not defined

In [105]:
≈  # alias for the `isapprox` function

isapprox (generic function with 15 methods)

In [106]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=atol>0 ? 0 : √eps, atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= max(atol, rtol*max(norm(x), norm(y)))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, if an `atol > 0` is not specified, `rtol` defaults to the square root of [`eps`](@ref) of the type of `x` or `y`, whichever is bigger (least precise). This corresponds to requiring equality of about half of the significand digits. Otherwise, e.g. for integer arguments or if an `atol > 0` is supplied, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

Note that `x ≈ 0` (i.e., comparing to zero with the default tolerances) is equivalent to `x == 0` since the default `atol` is `0`.  In such cases, you should either supply an appropriate `atol` (or use `norm(x) ≤ atol`) or rearrange your code (e.g. use `x ≈ y` rather than `x - y ≈ 0`).   It is not possible to pick a nonzero `atol` automatically because it depends on the overall scaling (the "units") of your problem: for example, in `x - y ≈ 0`, `atol=1e-9` is an absurdly small tolerance if `x` is the [radius of the Earth](https://en.wikipedia.org/wiki/Earth_radius) in meters, but an absurdly large tolerance if `x` is the [radius of a Hydrogen atom](https://en.wikipedia.org/wiki/Bohr_radius) in meters.

# Examples

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true

julia> 1e-10 ≈ 0
false

julia> isapprox(1e-10, 0, atol=1e-8)
true
```

---

```
isapprox(x::FixedPoint, y::FixedPoint; rtol=0, atol=max(eps(x), eps(y)))
```

For FixedPoint numbers, the default criterion is that `x` and `y` differ by no more than `eps`, the separation between adjacent fixed-point numbers.


We can now benchmark the C code directly from Julia:

In [107]:
c_bench = @benchmark c_sum($a)

UndefVarError: UndefVarError: c_sum not defined

In [108]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

UndefVarError: UndefVarError: c_bench not defined

In [109]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

UndefVarError: UndefVarError: c_bench not defined

In [110]:
using Plots
gr()

Plots.GRBackend()

In [111]:
using Statistics # bring in statistical support for standard deviations
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")
# # 2. C with -ffast-math

UndefVarError: UndefVarError: c_bench not defined

In [112]:
# If we allow C to re-arrange the floating point operations, then it'll vectorize with SIMD (single instruction, multiple data) instructions.
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

Base.IOError: IOError: could not spawn `gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o 'C:\Users\Guru\AppData\Local\Temp\jl_4EAD.tmp.dll' -`: no such file or directory (ENOENT)

In [113]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

UndefVarError: UndefVarError: c_sum_fastmath not defined

In [114]:
d["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds

UndefVarError: UndefVarError: c_fastmath_bench not defined

# 3. Python's built in `sum`

The `PyCall` package provides a Julia interface to Python:

In [115]:
# using Pkg; Pkg.add("PyCall")
using PyCall

ArgumentError: ArgumentError: Package PyCall not found in current path:
- Run `import Pkg; Pkg.add("PyCall")` to install the PyCall package.


In [116]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

UndefVarError: UndefVarError: pybuiltin not defined

In [117]:
pysum(a)

UndefVarError: UndefVarError: pysum not defined

In [118]:
pysum(a) ≈ sum(a)

UndefVarError: UndefVarError: pysum not defined

In [119]:
py_list_bench = @benchmark $pysum($a)

UndefVarError: UndefVarError: pysum not defined

In [120]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

UndefVarError: UndefVarError: py_list_bench not defined

# 4. Python: `numpy`

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [121]:
# using Pkg; Pkg.add("Conda")
using Conda

ArgumentError: ArgumentError: Package Conda not found in current path:
- Run `import Pkg; Pkg.add("Conda")` to install the Conda package.


In [122]:
# Conda.add("numpy")

In [123]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

UndefVarError: UndefVarError: pyimport not defined

In [124]:
numpy_sum(a)

UndefVarError: UndefVarError: numpy_sum not defined

In [125]:
numpy_sum(a) ≈ sum(a)

UndefVarError: UndefVarError: numpy_sum not defined

In [126]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

UndefVarError: UndefVarError: py_numpy_bench not defined

# 5. Python, hand-written

In [127]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

LoadError: UndefVarError: @py_str not defined

In [128]:
py_hand = @benchmark $sum_py($a)

UndefVarError: UndefVarError: sum_py not defined

In [129]:
sum_py(a)

UndefVarError: UndefVarError: sum_py not defined

In [130]:
sum_py(a) ≈ sum(a)

UndefVarError: UndefVarError: sum_py not defined

In [131]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

UndefVarError: UndefVarError: py_hand not defined

# 6. Julia (built-in)

## Written directly in Julia, not in C!

In [132]:
@which sum(a)

sum(a::AbstractArray) in Base at reducedim.jl:645

In [133]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.091 ms (0.00% GC)
  median time:      7.372 ms (0.00% GC)
  mean time:        7.679 ms (0.00% GC)
  maximum time:     10.815 ms (0.00% GC)
  --------------
  samples:          650
  evals/sample:     1

In [134]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 1 entry:
  "Julia built-in" => 7.0912

# 7. Julia (hand-written)

In [135]:
function mysum(A)
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [136]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     25.462 ms (0.00% GC)
  median time:      25.527 ms (0.00% GC)
  mean time:        25.702 ms (0.00% GC)
  maximum time:     28.492 ms (0.00% GC)
  --------------
  samples:          195
  evals/sample:     1

In [137]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "Julia hand-written" => 25.4622
  "Julia built-in"     => 7.0912

# 8. Julia (hand-written w. simd)

In [138]:
function mysum_simd(A)
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

mysum_simd (generic function with 1 method)

In [139]:
j_bench_hand_simd = @benchmark mysum_simd($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.062 ms (0.00% GC)
  median time:      7.292 ms (0.00% GC)
  mean time:        7.398 ms (0.00% GC)
  maximum time:     9.143 ms (0.00% GC)
  --------------
  samples:          675
  evals/sample:     1

In [140]:
mysum_simd(a)

5.001145501676659e6

In [141]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "Julia hand-written simd" => 7.0618
  "Julia hand-written"      => 25.4622
  "Julia built-in"          => 7.0912

# Summary

In [142]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia hand-written simd.....7.1
Julia built-in..............7.1
Julia hand-written.........25.5
